In [1]:
!pip install pytorch_tabnet
!pip install tableone
!pip install catboost

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 69.4 MB/s eta 0:00:0000:0100:01
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [pytorch_tabnet]m [torch]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tableone]


## DataLoad

In [63]:
import pandas as pd
import numpy as np
nanji = pd.read_csv('../data/processed/center_season/nanji/난지_merged.csv', encoding='utf-8-sig')

nanji.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103 entries, 0 to 3102
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   날짜             3103 non-null   object 
 1   요일             3103 non-null   object 
 2   공휴일            3103 non-null   int64  
 3   목욕장업           3103 non-null   int64  
 4   세탁업            3103 non-null   int64  
 5   수영장업           3103 non-null   int64  
 6   종합체육시설업        3103 non-null   int64  
 7   체력단련장업         3103 non-null   int64  
 8   하천             3103 non-null   float64
 9   생활인구           3103 non-null   float64
 10  불쾌지수(DI)       3103 non-null   float64
 11  불쾌지수등급         3103 non-null   object 
 12  일_일강수량(mm)     3103 non-null   float64
 13  일_최저기온(°C)     3103 non-null   float64
 14  일_평균기온(°C)     3103 non-null   float64
 15  일_최고기온(°C)     3103 non-null   float64
 16  일_평균풍속(m/s)    3103 non-null   float64
 17  일_최대순간풍속(m/s)  3103 non-null   float64
 18  최저습도(%) 

In [64]:
nanji['날짜'] = pd.to_datetime(nanji['날짜'])
nanji = nanji.sort_values('날짜').reset_index(drop=True)
# 난지 6월 데이터 없음 -> 다 0으로 되어 있으니까 제거 
nanji = nanji[nanji["날짜"] < "2025-06-01"] 

nanji['월'] = nanji['날짜'].dt.month #월

def add_cyclical_features(df, date_col='날짜'):
    d = df[date_col].dt

    # 요일
    dow = d.weekday
    df['요일_sin'] = np.sin(2*np.pi*dow/7)
    df['요일_cos'] = np.cos(2*np.pi*dow/7)

    # 연중일
    doy = d.dayofyear
    df['연중일_sin'] = np.sin(2*np.pi*doy/365.25)
    df['연중일_cos'] = np.cos(2*np.pi*doy/365.25)
    return df
nanji = add_cyclical_features(nanji)

nanji['요일_숫자'] = nanji['날짜'].dt.weekday # 0:월요일, 1:화요일, ... , 6:일요일 -> 요일 숫자로

# 계절, 불쾌지수등급 숫자로
season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
nanji['계절'] = nanji['계절'].map(season_map).astype('Int64')
nanji['불쾌지수등급'] = nanji['불쾌지수등급'].map(discomfort_map).astype('Int64')

# 타겟
nanji['합계_1일후'] = nanji['합계'].shift(-1)
nanji['합계_2일후'] = nanji['합계'].shift(-2)

In [65]:
nanji.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3073 entries, 0 to 3072
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   날짜             3073 non-null   datetime64[ns]
 1   요일             3073 non-null   object        
 2   공휴일            3073 non-null   int64         
 3   목욕장업           3073 non-null   int64         
 4   세탁업            3073 non-null   int64         
 5   수영장업           3073 non-null   int64         
 6   종합체육시설업        3073 non-null   int64         
 7   체력단련장업         3073 non-null   int64         
 8   하천             3073 non-null   float64       
 9   생활인구           3073 non-null   float64       
 10  불쾌지수(DI)       3073 non-null   float64       
 11  불쾌지수등급         3073 non-null   Int64         
 12  일_일강수량(mm)     3073 non-null   float64       
 13  일_최저기온(°C)     3073 non-null   float64       
 14  일_평균기온(°C)     3073 non-null   float64       
 15  일_최고기온(°C)     3073 non-nu

In [66]:
nanji = nanji.drop(['날짜', '요일','1처리장', '2처리장', '정화조', '중계펌프장'], axis=1)
nanji = nanji.dropna()
nanji.shape

(3071, 30)

In [67]:
nanji.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3071 entries, 0 to 3070
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   공휴일            3071 non-null   int64  
 1   목욕장업           3071 non-null   int64  
 2   세탁업            3071 non-null   int64  
 3   수영장업           3071 non-null   int64  
 4   종합체육시설업        3071 non-null   int64  
 5   체력단련장업         3071 non-null   int64  
 6   하천             3071 non-null   float64
 7   생활인구           3071 non-null   float64
 8   불쾌지수(DI)       3071 non-null   float64
 9   불쾌지수등급         3071 non-null   Int64  
 10  일_일강수량(mm)     3071 non-null   float64
 11  일_최저기온(°C)     3071 non-null   float64
 12  일_평균기온(°C)     3071 non-null   float64
 13  일_최고기온(°C)     3071 non-null   float64
 14  일_평균풍속(m/s)    3071 non-null   float64
 15  일_최대순간풍속(m/s)  3071 non-null   float64
 16  최저습도(%)        3071 non-null   float64
 17  평균습도(%)        3071 non-null   float64
 18  최고습도(%)      

In [68]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = nanji.drop(['합계','합계_1일후', '합계_2일후'], axis=1)
y = nanji['합계']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

(2763, 27) (308, 27) (2763,) (308,)


In [69]:
'''from sklearn.metrics import mean_squared_error, r2_score
from pytorch_tabnet.tab_model import TabNetRegressor
import torch

# 6. TabNetRegressor
tabnet = TabNetRegressor(
    n_d=10, n_a=10, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":20, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax',
    verbose=1
)

# numpy 변환
X_train_np = X_train_scaled.astype(np.float32)
X_test_np  = X_test_scaled.astype(np.float32)
y_train_np = y_train.to_numpy().reshape(-1, 1).astype(np.float32)  
y_test_np  = y_test.to_numpy().reshape(-1, 1).astype(np.float32)   

tabnet.fit(
    X_train=X_train_np, y_train=y_train_np,
    eval_set=[(X_test_np, y_test_np)],
    eval_metric=["rmse"],
    max_epochs=100,
    patience=5,
    batch_size=32,
    virtual_batch_size=16,
    num_workers=0,
    drop_last=False
)

y_pred_tabnet = tabnet.predict(X_test_np).squeeze()
results = {}
results["TabNet"] = {
    "model": tabnet,
    "rmse": np.sqrt(mean_squared_error(y_test_np, y_pred_tabnet)),
    "r2": r2_score(y_test_np, y_pred_tabnet),
    "y_pred": y_pred_tabnet
}

# 7. 결과 출력
for name, res in results.items():
    print(f"{name}: RMSE={res['rmse']:.2f}, R²={res['r2']:.4f}")
'''

'from sklearn.metrics import mean_squared_error, r2_score\nfrom pytorch_tabnet.tab_model import TabNetRegressor\nimport torch\n\n# 6. TabNetRegressor\ntabnet = TabNetRegressor(\n    n_d=10, n_a=10, n_steps=5,\n    gamma=1.5, n_independent=2, n_shared=2,\n    optimizer_fn=torch.optim.Adam,\n    optimizer_params=dict(lr=2e-2),\n    scheduler_params={"step_size":20, "gamma":0.9},\n    scheduler_fn=torch.optim.lr_scheduler.StepLR,\n    mask_type=\'entmax\',\n    verbose=1\n)\n\n# numpy 변환\nX_train_np = X_train_scaled.astype(np.float32)\nX_test_np  = X_test_scaled.astype(np.float32)\ny_train_np = y_train.to_numpy().reshape(-1, 1).astype(np.float32)  \ny_test_np  = y_test.to_numpy().reshape(-1, 1).astype(np.float32)   \n\ntabnet.fit(\n    X_train=X_train_np, y_train=y_train_np,\n    eval_set=[(X_test_np, y_test_np)],\n    eval_metric=["rmse"],\n    max_epochs=100,\n    patience=5,\n    batch_size=32,\n    virtual_batch_size=16,\n    num_workers=0,\n    drop_last=False\n)\n\ny_pred_tabnet = t

In [ ]:

import numpy as np
import pandas as pd
import math, time

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from pytorch_tabnet.tab_model import TabNetRegressor
import torch

# 1) 데이터 분리 
TARGET = '합계_1일후'
X = nanji.drop(columns=[c for c in ['합계_1일후','합계_2일후'] if c in nanji.columns]).copy()
y = nanji[TARGET].copy()

n = len(X)
test_n = int(math.ceil(n * 0.10))
train_n = n - test_n

X_train, X_test = X.iloc[:train_n, :].copy(), X.iloc[train_n:, :].copy()
y_train, y_test = y.iloc[:train_n].copy(), y.iloc[train_n:].copy()

print(f"[SPLIT] train={X_train.shape}, test={X_test.shape}")

# 2) 스케일링 (누수 방지: train으로만 fit)
x_scaler = StandardScaler()
X_train_s = x_scaler.fit_transform(X_train)
X_test_s  = x_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_s = y_scaler.fit_transform(y_train.values.reshape(-1,1)).astype(np.float32)  # (n,1)
y_test_s  = y_scaler.transform(y_test.values.reshape(-1,1)).astype(np.float32)      # (m,1)

# TabNet 입력 dtype/shape
X_train_np = X_train_s.astype(np.float32)
X_test_np  = X_test_s.astype(np.float32)

# 3) TabNet 설정
tabnet = TabNetRegressor(
    n_d=32, n_a=32, n_steps=4,
    gamma=1, n_independent=2, n_shared=2,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3, weight_decay=1e-5),
    scheduler_fn=None,      # 우선 스케줄러 OFF로 수렴 확인
    mask_type='entmax',                # 엔트맥스가 일반적으로 안정적
    verbose=1,
    seed=42
)

t0 = time.time()
tabnet.fit(
    X_train=X_train_np,
    y_train=y_train_s,                    # <-- (n,1) 2D
    eval_set=[(X_test_np, y_test_s)],     # <-- (m,1) 2D
    eval_metric=["rmse"],                 # 표준화된 y 기준 RMSE
    max_epochs=500,
    patience=50,
    batch_size=512,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)
print(f"TabNet training time: {time.time()-t0:.1f}s")

# 4) 예측 & 역변환 평가 
y_pred_s = tabnet.predict(X_test_np).squeeze()                 # (m,)
y_pred = y_scaler.inverse_transform(y_pred_s.reshape(-1,1)).ravel()

y_true = y_test.values
mse  = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)   # 버전 독립 RMSE
mae  = mean_absolute_error(y_true, y_pred)
r2   = r2_score(y_true, y_pred)

print(f"[TabNet] MAE={mae:.3f}  RMSE={rmse:.3f}  R²={r2:.4f}")

# 5) 기준선과 비교 (원 스케일)
y_test_arr = y_true
# 어제값(naive-1): 테스트 내 시프트로 근사 비교 (첫 값 보정)
y_na1 = np.roll(y_test_arr, 1); y_na1[0] = y_test_arr[0]
# 7일 전(week naive): 주간 패턴 근사
y_wk  = np.roll(y_test_arr, 7); y_wk[:7] = y_test_arr[:7]

rmse_na1 = np.sqrt(mean_squared_error(y_test_arr, y_na1)); r2_na1 = r2_score(y_test_arr, y_na1)
rmse_wk  = np.sqrt(mean_squared_error(y_test_arr, y_wk));  r2_wk  = r2_score(y_test_arr, y_wk)

print(f"[Naive-1]   RMSE={rmse_na1:.3f}  R²={r2_na1:.4f}")
print(f"[Week-Naive]RMSE={rmse_wk:.3f}  R²={r2_wk:.4f}")


[SPLIT] train=(2763, 28), test=(308, 28)


/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 4.75102 | val_0_rmse: 1.84898 |  0:00:00s
epoch 1  | loss: 3.30284 | val_0_rmse: 1.20244 |  0:00:00s
epoch 2  | loss: 2.27439 | val_0_rmse: 1.01814 |  0:00:00s
epoch 3  | loss: 1.7867  | val_0_rmse: 1.00781 |  0:00:00s
epoch 4  | loss: 1.60676 | val_0_rmse: 0.99834 |  0:00:00s
epoch 5  | loss: 1.53675 | val_0_rmse: 1.01613 |  0:00:01s
epoch 6  | loss: 1.31314 | val_0_rmse: 1.05899 |  0:00:01s
epoch 7  | loss: 1.17958 | val_0_rmse: 1.06617 |  0:00:01s
epoch 8  | loss: 1.05648 | val_0_rmse: 1.03472 |  0:00:01s
epoch 9  | loss: 1.04824 | val_0_rmse: 1.05552 |  0:00:01s
epoch 10 | loss: 0.94672 | val_0_rmse: 1.06754 |  0:00:02s
epoch 11 | loss: 0.84219 | val_0_rmse: 0.96109 |  0:00:02s
epoch 12 | loss: 0.86821 | val_0_rmse: 0.87624 |  0:00:02s
epoch 13 | loss: 0.81331 | val_0_rmse: 0.87541 |  0:00:02s
epoch 14 | loss: 0.76561 | val_0_rmse: 0.88799 |  0:00:02s
epoch 15 | loss: 0.7455  | val_0_rmse: 0.86846 |  0:00:02s
epoch 16 | loss: 0.68841 | val_0_rmse: 0.82915 |  0:00:0

/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:

import numpy as np
import pandas as pd
import math, time

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from pytorch_tabnet.tab_model import TabNetRegressor
import torch

# 데이터 분리
TARGET = '합계'
X = nanji.drop(columns=[c for c in ['합계', '합계_1일후','합계_2일후'] if c in nanji.columns]).copy()
y = nanji[TARGET].copy()

n = len(X)
test_n = int(math.ceil(n * 0.10))
train_n = n - test_n

X_train, X_test = X.iloc[:train_n, :].copy(), X.iloc[train_n:, :].copy()
y_train, y_test = y.iloc[:train_n].copy(), y.iloc[train_n:].copy()

print(f"[SPLIT] train={X_train.shape}, test={X_test.shape}")

# 스케일링
x_scaler = StandardScaler()
X_train_s = x_scaler.fit_transform(X_train)
X_test_s  = x_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_s = y_scaler.fit_transform(y_train.values.reshape(-1,1)).astype(np.float32)  # (n,1)
y_test_s  = y_scaler.transform(y_test.values.reshape(-1,1)).astype(np.float32)      # (m,1)

# TabNet 입력 dtype/shape
X_train_np = X_train_s.astype(np.float32)
X_test_np  = X_test_s.astype(np.float32)

# TabNet 설정
tabnet = TabNetRegressor(
    n_d=32, n_a=32, n_steps=4,
    gamma=1.5, n_independent=2, n_shared=2,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3, weight_decay=1e-5),
    scheduler_fn=None,                 # 우선 스케줄러 OFF로 수렴 확인
    mask_type='entmax',                # 엔트맥스가 일반적으로 안정적
    verbose=1,
    seed=42
)

t0 = time.time()
tabnet.fit(
    X_train=X_train_np,
    y_train=y_train_s,                    # <-- (n,1) 2D
    eval_set=[(X_test_np, y_test_s)],     # <-- (m,1) 2D
    eval_metric=["rmse"],                 # 표준화된 y 기준 RMSE
    max_epochs=500,
    patience=50,
    batch_size=512,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)
print(f"TabNet training time: {time.time()-t0:.1f}s")

y_pred_s = tabnet.predict(X_test_np).squeeze()                 # (m,)
y_pred = y_scaler.inverse_transform(y_pred_s.reshape(-1,1)).ravel()

y_true = y_test.values
mse  = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)   # 버전 독립 RMSE
mae  = mean_absolute_error(y_true, y_pred)
r2   = r2_score(y_true, y_pred)

print(f"[TabNet] MAE={mae:.3f}  RMSE={rmse:.3f}  R²={r2:.4f}")

y_test_arr = y_true
# 어제값(naive-1): 테스트 내 시프트로 근사 비교 (첫 값 보정)
y_na1 = np.roll(y_test_arr, 1); y_na1[0] = y_test_arr[0]
# 7일 전(week naive): 주간 패턴 근사
y_wk  = np.roll(y_test_arr, 7); y_wk[:7] = y_test_arr[:7]

rmse_na1 = np.sqrt(mean_squared_error(y_test_arr, y_na1)); r2_na1 = r2_score(y_test_arr, y_na1)
rmse_wk  = np.sqrt(mean_squared_error(y_test_arr, y_wk));  r2_wk  = r2_score(y_test_arr, y_wk)

print(f"[Naive-1]   RMSE={rmse_na1:.3f}  R²={r2_na1:.4f}")
print(f"[Week-Naive]RMSE={rmse_wk:.3f}  R²={r2_wk:.4f}")


[SPLIT] train=(2763, 27), test=(308, 27)


/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 3.61672 | val_0_rmse: 1.75849 |  0:00:00s
epoch 1  | loss: 2.71206 | val_0_rmse: 1.26097 |  0:00:00s
epoch 2  | loss: 2.13977 | val_0_rmse: 1.07348 |  0:00:00s
epoch 3  | loss: 1.75452 | val_0_rmse: 1.03323 |  0:00:00s
epoch 4  | loss: 1.48026 | val_0_rmse: 0.9804  |  0:00:00s
epoch 5  | loss: 1.26208 | val_0_rmse: 0.92909 |  0:00:01s
epoch 6  | loss: 1.19113 | val_0_rmse: 0.88741 |  0:00:01s
epoch 7  | loss: 1.08766 | val_0_rmse: 0.89136 |  0:00:01s
epoch 8  | loss: 0.98387 | val_0_rmse: 0.8518  |  0:00:01s
epoch 9  | loss: 0.95525 | val_0_rmse: 0.80904 |  0:00:01s
epoch 10 | loss: 0.88944 | val_0_rmse: 0.78601 |  0:00:02s
epoch 11 | loss: 0.80558 | val_0_rmse: 0.83771 |  0:00:02s
epoch 12 | loss: 0.77179 | val_0_rmse: 0.81076 |  0:00:02s
epoch 13 | loss: 0.77625 | val_0_rmse: 0.83842 |  0:00:02s
epoch 14 | loss: 0.71271 | val_0_rmse: 0.80068 |  0:00:02s
epoch 15 | loss: 0.70846 | val_0_rmse: 0.79525 |  0:00:02s
epoch 16 | loss: 0.6908  | val_0_rmse: 0.75433 |  0:00:0

/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


## TableOne


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
import pandas as pd
import torch

# 1. 데이터 생성
target = data_clean['price']
x_data_clean = data_clean.drop(['price'], axis=1)

# 2. train-test 분할
X_train, X_test, y_train, y_test = train_test_split(x_data_clean, target, test_size=0.2, random_state=42)

# 3. 스케일링
x_scaler = StandardScaler()
y_scaler = StandardScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled = x_scaler.transform(X_test)
Y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
Y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))


# 6. TabNetRegressor
tabnet = TabNetRegressor(
    n_d=10, n_a=10, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax',
    verbose=1
)

# numpy 변환
X_train_np = X_train_scaled
X_test_np = X_test_scaled
y_train_np = Y_train_scaled.reshape(-1, 1)
y_test_np = Y_test_scaled.reshape(-1, 1)


tabnet.fit(
    X_train=X_train_np, y_train=y_train_np,
    eval_set=[(X_test_np, y_test_np)],
    eval_metric=["rmse"],
    max_epochs=100,
    patience=5,
    batch_size=32,
    virtual_batch_size=16,
    num_workers=0,
    drop_last=False
)

y_pred_tabnet = tabnet.predict(X_test_np).squeeze()
results = {}
results["TabNet"] = {
    "model": tabnet,
    "rmse": np.sqrt(mean_squared_error(y_test_np, y_pred_tabnet)),
    "r2": r2_score(y_test_np, y_pred_tabnet),
    "y_pred": y_pred_tabnet
}

# 7. 결과 출력
for name, res in results.items():
    print(f"{name}: RMSE={res['rmse']:.2f}, R²={res['r2']:.4f}")


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.82031 | val_0_rmse: 0.79211 |  0:00:03s
epoch 1  | loss: 1.29141 | val_0_rmse: 0.81296 |  0:00:05s
epoch 2  | loss: 1.21116 | val_0_rmse: 0.83193 |  0:00:06s
epoch 3  | loss: 1.06765 | val_0_rmse: 0.80885 |  0:00:08s
epoch 4  | loss: 1.01738 | val_0_rmse: 0.77695 |  0:00:09s
epoch 5  | loss: 1.00513 | val_0_rmse: 0.81134 |  0:00:10s
epoch 6  | loss: 1.1137  | val_0_rmse: 0.76654 |  0:00:11s
epoch 7  | loss: 1.01181 | val_0_rmse: 0.74934 |  0:00:12s
epoch 8  | loss: 0.99809 | val_0_rmse: 0.78254 |  0:00:14s
epoch 9  | loss: 0.98623 | val_0_rmse: 0.7726  |  0:00:16s
epoch 10 | loss: 0.97368 | val_0_rmse: 0.74438 |  0:00:18s
epoch 11 | loss: 1.04397 | val_0_rmse: 0.73356 |  0:00:20s
epoch 12 | loss: 1.00513 | val_0_rmse: 0.74566 |  0:00:21s
epoch 13 | loss: 1.00761 | val_0_rmse: 0.75986 |  0:00:22s
epoch 14 | loss: 0.99114 | val_0_rmse: 0.73782 |  0:00:22s
epoch 15 | loss: 0.95215 | val_0_rmse: 0.76794 |  0:00:23s
epoch 16 | loss: 0.96789 | val_0_rmse: 0.75083 |  0:00:2

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet: RMSE=0.73, R²=0.1064
